In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import convNN as conv
import tensorflow as tf



path = "/media/jahan/solo/"
os.chdir(path)

labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
print(len(labels))
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
print(len(labels))


all_image_paths = labels.Path.values

labels = [1 if x == 1 else 0 for x in labels["No Finding"]]

img_path = all_image_paths[0]
print(img_path)

img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")
img_tensor = tf.image.decode_jpeg(img_raw,channels=1)
print(img_tensor.shape)
print(img_tensor.dtype)


def preprocess_image(image):
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path): # load from path and return tensor
    image = tf.io.read_file(path)
    return preprocess_image(image)

import matplotlib.pyplot as plt

image_path = all_image_paths[0]
label = labels[0]

plt.imshow(load_and_preprocess_image(img_path).numpy())
plt.grid(False)
print()

W0813 19:53:02.644839 139737634625344 deprecation_wrapper.py:119] From /home/jahan/Documents/CXR_CNN/algorithms/autoencoder.py:11: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



223414
191027
CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg
<tf.Tensor: id=1, shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x...
(320, 389, 1)
<dtype: 'uint8'>



In [4]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import convNN as conv
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot

path = "/media/jahan/solo/"
os.chdir(path)

labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
print(len(labels))
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
print(len(labels))


all_image_paths = labels.Path.values

labels = [1 if x == 1 else 0 for x in labels["No Finding"]]
encode = onehot()

labels = encode.fit_transform(np.array(labels).reshape((-1,1))).toarray()
print(labels[:5])

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


save_path = "/home/jahan/Documents/CXR_CNN/output/"


resnet = conv(image_paths=all_image_paths, labels=labels, save_path=save_path)
resnet.build_dataset(batch_size=32)
resnet.build()

223414
191027
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


/home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 16) 416         input_1[0][0]                    
__________________________________________________________________________________________________
12c (BatchNormalization)        (None, 256, 256, 16) 64          conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           12c[0][0]                        
______________________________________________________________________________________________

In [5]:
resnet.train(100)

Epoch 1/100


W0813 20:07:16.257368 139773784188736 deprecation.py:323] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: logits and labels must have the same first dimension, got logits shape [32,2] and labels shape [64]
	 [[{{node loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]
  (1) Invalid argument: logits and labels must have the same first dimension, got logits shape [32,2] and labels shape [64]
	 [[{{node loss/dense_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]
	 [[Func/Adam/gradients/bn2c_branch2a/cond_grad/If/then/_330/input/_1000/_454]]
0 successful operations.
0 derived errors ignored. [Op:__inference_keras_scratch_graph_7277]

In [ ]:
prediction = resnet.autoencoder.predict(data['a'][1].reshape((1,256,256,1)))

%matplotlib notebook
from matplotlib import pyplot as plt

plt.imshow(prediction.reshape((256,256))*255, cmap='gray')
plt.show()

In [ ]:
prediction